In [40]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
import re

pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [41]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [42]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [43]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
adwile_banner
/02_regular_reports


In [44]:
data_link = download_response.content

In [45]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [46]:
sheet_names.sheet_names

['Sheet1']

In [33]:
report_type = 'test'

In [82]:
tmp_banner_dict = {}
cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
df = pd.read_excel(BytesIO(data_link), sheet_name='Sheet1', usecols=cols_range, header=None)
df = df.fillna('')

start_index_list = list(df[df[0]=='Дата'].index)  # берем индекс начала таблицы с данными
end_index_list = list(df[df[0].str.contains('Итого')].index) # берем индекс окончания таблицы с данными

df.columns = df.iloc[start_index_list[0]].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
df = df.rename(columns={'дата': 'date', 'клики': 'clicks', 'показы': 'impressions', 'охват': 'reach', 'creo': 'product'}) 
df = df[['date', 'impressions', 'clicks', 'reach', 'product']] # оставляем только нужные поля
for i in range(len(start_index_list)):
    start_index = start_index_list[i] # получаем индкс начала конкретной таблицы
    end_index =  end_index_list[i]  # получаем индкс окончания конкретной таблицы

    df_tmp = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны
    df_tmp = df_tmp.fillna('')
    df_tmp['source'] = 'adwile' #добавляем название источника
    df_tmp['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
    df_tmp['date'] = pd.to_datetime(df_tmp['date']).dt.date  # приводим в формат даты

    df_tmp['report_type'] = report_type
    tmp_banner_dict[i] = df_tmp
    


In [83]:
tmp_banner_dict

{0: 0         date  impressions  clicks  reach product  source format_type  \
 1   2024-08-26        49652     132  19872   creo1  adwile      banner   
 2   2024-08-27        84009     127  33615   creo1  adwile      banner   
 3   2024-08-28        65622     129  26260   creo1  adwile      banner   
 4   2024-09-22        28989      67  11607   creo1  adwile      banner   
 5   2024-09-23        59357     115  23754   creo1  adwile      banner   
 6   2024-09-24        52516     105  21017   creo1  adwile      banner   
 7   2024-09-25        82753     107  33112   creo1  adwile      banner   
 8   2024-09-26        56386     102  22565   creo1  adwile      banner   
 9   2024-09-27        73614      94  29457   creo1  adwile      banner   
 10  2024-09-28        87158      98  34874   creo1  adwile      banner   
 11  2024-09-29        69735      76  27625   creo1  adwile      banner   
 
 0  report_type  
 1         test  
 2         test  
 3         test  
 4         test  
 5   

In [59]:
df.head(30)

,дата,клики,показы,охват,creo
0,Дата,Клики,Показы,Охват,creo
1,2024-08-26,132,49652,19872,creo1
2,2024-08-27,127,84009,33615,creo1
3,2024-08-28,129,65622,26260,creo1
4,2024-09-22,67,28989,11607,creo1
5,2024-09-23,115,59357,23754,creo1
6,2024-09-24,105,52516,21017,creo1
7,2024-09-25,107,82753,33112,creo1
8,2024-09-26,102,56386,22565,creo1
9,2024-09-27,94,73614,29457,creo1


In [62]:

for name in sheet_names.sheet_names[1:2]:
    # передаем название листа для парсинга, диапазон колонок, которые нам нужны и заранее подготовленный списко названий полей
    df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, header=None) 
    df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить

    base_text = df[0][3].lower().strip() # Название креатива(продукта) в столбике А в 4 строке - должно начинаться со слова креатив
    base_text = re.sub('[_!;:()-]+', ' ', base_text ) # убираем из названия креатива лишние символы
    base_text = re.sub(' +', ' ', base_text) # если встречаются пробелы больше 1 подряд, то убираем лишние
    
    start_index = str(base_text).find('креатив') # Забираем индекс, с которого начинается название креатива
    product_name = str(base_text)[start_index:] # сохраняем название креатива

    start_index_list = list(df[df[0]=='Дата'].index)  # берем индекс начала таблицы с данными
    end_index_list = list(df[df[0]=='Всего'].index) # берем индекс окончания таблицы с данными
    
    df.columns = df.iloc[start_index].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
     # привоодим названия к единому стандарту
    df = df.rename(columns={'дата': 'date', 'охват': 'reach', 'показы': 'impressions', 'видимые показы': 'views', 'переходы': 'clicks'}) 
    df = df[['date', 'reach', 'impressions', 'views', 'clicks']] # оставляем только нужные поля

    

    df = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны

In [64]:
start_index

0

In [63]:
df.head(7)

,дата,клики,показы,охват,creo
0,Дата,Клики,Показы,Охват,creo
1,2024-08-26,132,49652,19872,creo1
2,2024-08-27,127,84009,33615,creo1
3,2024-08-28,129,65622,26260,creo1
4,2024-09-22,67,28989,11607,creo1
5,2024-09-23,115,59357,23754,creo1
6,2024-09-24,105,52516,21017,creo1


In [37]:
product_name

'креатив 5 колесо обозрения'

In [ ]:
# источник Gnezdo
# тип отчета Баннерная реклама
# Функция для обработки статистики по дням для обычных размещений 
# 1. На первом листе находится общая статистика 
# 2. Статистика по креативам в разбивке по дням начинается со 2 листа
# 3. Каждый креатив на отдельном листе
# 4. Название креатива(продукта) в столбике А в 4 строке - должно начинаться со слова креатив - Например креатив называется - Креатив 3_с котом
# Что записано в строке: Национальная лотерея - Национальная лотерея_Гарантированно разыгрываем 100 млн рублей (Креатив 3_с котом) 
# 5. В столбике А обязательно должна находиться Дата
# 6. Таблица должна заканчиваться строкой итогов. В столбике А в этой строке должно быть слово Всего

def get_gnezdo_banner_report(data_link, report_type):
    tmp_banner_dict = {}
    
    cols_range = 'A:E' # задаем диапазон полей, которые нам нужны  
    sheet_names = pd.ExcelFile(BytesIO(data_link))
    # проходим через цикл по списку названий листов
    # sheets_list = sheet_names.sheet_names[1:]
    for name in sheet_names.sheet_names[1:]:
        # передаем название листа для парсинга, диапазон колонок, которые нам нужны и заранее подготовленный списко названий полей
        df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, header=None) 
        df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить

        base_text = df[0][3].lower().strip() # Название креатива(продукта) в столбике А в 4 строке - должно начинаться со слова креатив
        base_text = re.sub('[_!;:()-]+', ' ', base_text ) # убираем из названия креатива лишние символы
        base_text = re.sub(' +', ' ', base_text) # если встречаются пробелы больше 1 подряд, то убираем лишние

        start_index = str(base_text).find('креатив') # Забираем индекс, с которого начинается название креатива
        product_name = str(base_text)[start_index:] # сохраняем название креатива

        start_index = list(df[df[0]=='Дата'].index)[0]  # берем индекс начала таблицы с данными
        end_index = list(df[df[0]=='Всего'].index)[0] # берем индекс окончания таблицы с данными

        df.columns = df.iloc[start_index].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
        df = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны
         # привоодим названия к единому стандарту
        df = df.rename(columns={'дата': 'date', 'охват': 'reach', 'показы': 'impressions', 'видимые показы': 'views', 'переходы': 'clicks'}) 
        df = df[['date', 'reach', 'impressions', 'views', 'clicks']] # оставляем только нужные поля
    
        df['source'] = 'gnezdo' #добавляем название источника
        df['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
        df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date().strftime('%Y-%m-%d'))# приводим в формат даты
        
        df['product'] = str(product_name).lower().strip() # добавляем статичное поле с названием продукта
        # df['product'] = df['product'].str.lower().str.strip()
        df['report_type'] = report_type
        # сохраняем датаФрейм во временный словарь 
        # ключ - это название продукта (15s, 6s и тд)
        tmp_banner_dict[name] = df

    return pd.concat(tmp_banner_dict, ignore_index=True)